<a href="https://colab.research.google.com/github/crazimon/github-slideshow/blob/master/Entity_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is intended to allow a user to run entity analysis on three URLs and understand differences in central topics covered by pieces of content. It is a tutorial implementation of Google's NLP entity Analysis API. 

For more guidance on how to run this, see here: https://sashadagayev.com/systematically-analyze-your-content-vs-competitor-content-and-make-actionable-improvements/

To use this notebook - go to File > Save a Copy in Drive.


In [ ]:
import os
import json
import urllib3
import pandas as pd

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/nlp-colab.json" #change to the name of your credentials. These can be obtained here:https://cloud.google.com/natural-language/docs/quickstart

In [ ]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.cloud import language_v1
from google.cloud.language_v1 import enums

# Instantiates a client

client = language.LanguageServiceClient()

This is the default sentiment analysis method recommended by Google. It is currently set up to take in an HTML response from a page.

In [ ]:
def sample_analyze_entities(html_content):
    client = language_v1.LanguageServiceClient()

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.HTML #you can change this to be just text; doesn't have to be HTML.

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": html_content, "type": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_entities(document, encoding_type=encoding_type)
    return response


In [ ]:
def return_entity_dataframe(response):
  output = sample_analyze_entities(response.data)
  output_list = []
  for entity in output.entities:
    entity_dict = {}
    entity_dict['entity_name'] = entity.name
    entity_dict['entity_type'] = enums.Entity.Type(entity.type).name
    entity_dict['entity_salience('+response._request_url+')'] = entity.salience
    entity_dict['entity_number_of_mentions('+response._request_url+')'] = len(entity.mentions)
    output_list.append(entity_dict)
  json_entity_analysis = json.dumps(output_list)
  df = pd.read_json(json_entity_analysis)
  summed_df = df.groupby(['entity_name']).sum()
  summed_df.sort_values(by=['entity_salience('+response._request_url+')'], ascending=False)
  return summed_df



```
# This is formatted as code
```

# Swap out your URLs HERE!!!


In [ ]:
http = urllib3.PoolManager()
response1 = http.request('GET','https://en.wikipedia.org/wiki/Search_engine_optimization')
response2 = http.request('GET','http://mozseoclass.com/who-is-the-smartest-seo/')
response3 = http.request('GET','https://hookagency.com/who-is-the-smartest-seo-in-the-world/')
response4 = http.request('GET','https://hookagency.com/who-is-the-smartest-seo-in-the-world/')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
url1_analysis = return_entity_dataframe(response1)
url2_analysis = return_entity_dataframe(response2)
url3_analysis = return_entity_dataframe(response3)

These functions here join all of the entity results into one giant table so that it can be easier for the users to review results and compare. 

In [ ]:
url1and2 = url1_analysis.merge(url2_analysis,how='outer', left_on='entity_name', right_on="entity_name")
url1and2and3 = url1and2.merge(url3_analysis,how='outer', left_on='entity_name', right_on="entity_name")

In [ ]:
url1and2and3.sort_values(by=['entity_salience('+response1._request_url+')'], ascending=False)

,entity_salience(https://en.wikipedia.org/wiki/Search_engine_optimization),entity_number_of_mentions(https://en.wikipedia.org/wiki/Search_engine_optimization),entity_salience(http://mozseoclass.com/who-is-the-smartest-seo/),entity_number_of_mentions(http://mozseoclass.com/who-is-the-smartest-seo/),entity_salience(https://hookagency.com/who-is-the-smartest-seo-in-the-world/),entity_number_of_mentions(https://hookagency.com/who-is-the-smartest-seo-in-the-world/)
entity_name,,,,,,
Google,0.450027,72.0,0.037636,7.0,0.001241,6.0
search engines,0.082046,33.0,NaN,NaN,0.000236,1.0
Stats Show Google,0.050376,13.0,NaN,NaN,NaN,NaN
Wikipedia,0.033818,12.0,NaN,NaN,NaN,NaN
search engine,0.018523,13.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
winner,NaN,NaN,NaN,NaN,0.000073,1.0
wins,NaN,NaN,NaN,NaN,0.000412,2.0
world battle,NaN,NaN,NaN,NaN,0.000255,1.0


In [ ]:
url1and2and3.to_csv('x-entities.csv')